In [ ]:
using Pkg;
Pkg.activate("./")
Pkg.instantiate()

# Loading SOLPS output into IMAS data structure

In [ ]:
using SOLPS2IMAS: SOLPS2IMAS

In [ ]:
b2gmtry = "../sample/ITER_Lore_2296_00000/baserun/b2fgmtry"
b2output = "../sample/ITER_Lore_2296_00000/run_time_dep_EIRENE_jdl_to_ss_cont_sine2_2d_output/b2time.nc"
b2mn = "../sample/ITER_Lore_2296_00000/run_time_dep_EIRENE_jdl_to_ss_cont_sine2_2d_output/b2mn.dat"
fort = (
    "../sample/ITER_Lore_2296_00000/baserun/fort.33",
    "../sample/ITER_Lore_2296_00000/baserun/fort.34",
    "../sample/ITER_Lore_2296_00000/baserun/fort.35",
    )

In [ ]:
ids = SOLPS2IMAS.solps2imas(b2gmtry, b2output; b2mn=b2mn, fort=fort);

# Visualising some properties

In [ ]:
using GGDUtils: GGDUtils
using Plots

IMAS ids store mesh information for edge profiles in grid_ggd. There are options to have multiple space representaions but typically you will have only one space describing the SOLPS mesh

In [ ]:
grid_ggd = ids.edge_profiles.grid_ggd[1]; # First grid_ggd time slice. It is allowed to vary in time
space = grid_ggd.space[1]; # First space in this grid_ggd

### Plotting grid and subsets

In [ ]:
# Choose backend
gr()           # Fast and can save pdf
# plotlyjs()   # Use for interactive plot, can only save png

plot(space) # Simply plot the grid described in space, all common arguments to plot can be given here

# You can overlay any subset by giving a second argument
# Labels
plot!(space, GGDUtils.get_grid_subset(grid_ggd, "x_points"), markercolor=:chocolate1, label="X-point")
plot!(space, GGDUtils.get_grid_subset(grid_ggd, "core_cut"), linecolor=:red, linewidth=2, label="Core Cut")
plot!(space, GGDUtils.get_grid_subset(grid_ggd, "PFR_cut"), linecolor=:darkred, linewidth=2, label="PFR Cut")
plot!(space, GGDUtils.get_grid_subset(grid_ggd, "outer_throat"), linecolor=:limegreen, linewidth=2, label="Outer Throat")
plot!(space, GGDUtils.get_grid_subset(grid_ggd, "inner_throat"), linecolor=:darkgreen, linewidth=2, label="Inner Throat")
plot!(space, GGDUtils.get_grid_subset(grid_ggd, "outer_midplane"), linecolor=:cyan, linewidth=2, label="Outer midplane")
# plot!(space, GGDUtils.get_grid_subset(grid_ggd, "inner_midplane"), linecolor=:teal, linewidth=2, label="Inner midplane")
plot!(space, GGDUtils.get_grid_subset(grid_ggd, "outer_target"), linecolor=:royalblue1, linewidth=2, label="Outer target")
plot!(space, GGDUtils.get_grid_subset(grid_ggd, "inner_target"), linecolor=:navyblue, linewidth=2, label="Inner target")
plot!(space, GGDUtils.get_grid_subset(grid_ggd, "core_boundary"), linecolor=:fuchsia, linewidth=2, linestyle=:dash, label="Core boundary")
plot!(space, GGDUtils.get_grid_subset(grid_ggd, "separatrix"), linecolor=:purple4, linewidth=2, linestyle=:dash, label="Separatrix")
plot!(space, GGDUtils.get_grid_subset(grid_ggd, 101), markershape=:rect, markercolor=:royalblue1)
# plot!(space, GGDUtils.get_grid_subset(grid_ggd, 102), markershape=:rect, markercolor=:maroon)
plot!(space, GGDUtils.get_grid_subset(grid_ggd, 103), markershape=:diamond, markercolor=:fuchsia)
plot!(space, GGDUtils.get_grid_subset(grid_ggd, 104), markershape=:diamond, markercolor=:purple4)

# Legend is supressed unless asked for specifically
plot!(legend=true, left_margin=10Plots.pt)
# Default labels are subset.identifier.name but can be changed by providing a label argument


### Plotting 2D quantities as heatmaps

In [ ]:
# Choose backend
gr()           # Fast and can save pdf
# plotlyjs()   # Use for interactive plot, can only save png

n_e = GGDUtils.get_prop_with_grid_subset_index(ids.edge_profiles.ggd[1].electrons.density, -5)
plot(ids.edge_profiles.grid_ggd, n_e, colorbar_title="Electrons density / m^(-3)",
     left_margin=10Plots.pt)

You can overlap any grid on top of a quantity

In [ ]:

# Choose backend
gr()           # Fast and can save pdf
# plotlyjs()   # Use for interactive plot, can only save png

plot(ids.edge_profiles.grid_ggd, n_e) # Note default label in colorbar
plot!(space, GGDUtils.get_grid_subset(grid_ggd, 16), linecolor=:black, linewidth=2,
      linestyle=:solid, label="Separatix", legend=true, left_margin=10Plots.pt)

# Adding equilibrium data

In [ ]:
import SD4SOLPS: SD4SOLPS

In [ ]:
eqdsk = "../sample/ITER_Lore_2296_00000/EQDSK/g002296.00200"
SD4SOLPS.geqdsk_to_imas!(eqdsk, ids; set_time=0.2)

### Extrapole core profile

In [ ]:
SD4SOLPS.fill_in_extrapolated_core_profile!(ids, "electrons.density"; method="simple", cell_subset_idx=-5)
SD4SOLPS.fill_in_extrapolated_core_profile!(ids, "electrons.temperature"; method="simple", cell_subset_idx=-5)
# ... more profiles here as they become available in b2time

# Loading a synthetic diagnostic

In [ ]:
using SynthDiag: SynthDiag

### Add interferometer chord details using a json file

In [ ]:
SynthDiag.add_interferometer!(SynthDiag.default_ifo, ids; n_e_gsi=-5);

### Plotting the interferometer geometry on top of SOLPS mesh

In [ ]:
# Choose backend
gr()           # Fast and can save pdf
# plotlyjs()   # Use for interactive plot, can only save png

plot(space)
plot!(ids.interferometer) # Default plot_type is :los 
plot!(legend=true, left_margin=10Plots.pt)

You can provide custom length and thickness of mirror to be plotted and linewidth of the laser beams

In [ ]:
# Choose backend
gr()           # Fast and can save pdf
# plotlyjs()   # Use for interactive plot, can only save png

plot(space)
plot!(ids.interferometer, mirror_length=0.7, linewidth=4, mirror_thickness=0.2)
plot!(legend=true, left_margin = 10Plots.pt)

Or you can choose to omit the mirror

In [ ]:
# Choose backend
gr()           # Fast and can save pdf
# plotlyjs()   # Use for interactive plot, can only save png

plot(space)
plot!(ids.interferometer, mirror=false)
plot!(legend=true, left_margin=10Plots.pt)

You can plot a single channel as well. You can override the in-built channel name for the label.

In [ ]:
# Choose backend
gr()           # Fast and can save pdf
# plotlyjs()   # Use for interactive plot, can only save png

plot(space)
plot!(ids.interferometer.channel[1], label="Channel 1")
plot!(legend=true, left_margin=10Plots.pt)

### Plotting interferometer data vs time

 * Use plot_type=:n_e for integrated electron density data
 * Use plot_type=:n_e_average for averaged electron density data


In [ ]:
# Choose backend
gr()           # Fast and can save pdf
# plotlyjs()   # Use for interactive plot, can only save png

plot(ids.interferometer, plot_type=:n_e, left_margin=10Plots.pt)

In [ ]:
# Choose backend
gr()           # Fast and can save pdf
# plotlyjs()   # Use for interactive plot, can only save png

plot(ids.interferometer, plot_type=:n_e_average, left_margin=10Plots.pt)

Again, to plot an individual channel, just provide the channel with correct plot_type

In [ ]:
# Choose backend
gr()           # Fast and can save pdf
# plotlyjs()   # Use for interactive plot, can only save png

plot(ids.interferometer.channel[1], plot_type=:n_e_average, left_margin=10Plots.pt)

# Load Langmuir Probes

Same syntax as interferometer

In [ ]:
SynthDiag.add_langmuir_probes!(SynthDiag.default_lp, ids; n_e_gsi=-5);

 Data visualization recipes for langmuir probes have not been created yet and might not be created but you can still see them using built in plotting methodsan d knowledgeof IMAS data structure.

In [ ]:
gr()           # Fast and can save pdf
# plotlyjs()   # Use for interactive plot, can only save png

plot(ids.langmuir_probes.embedded[1].time, ids.langmuir_probes.embedded[1].n_e.data, label=ids.langmuir_probes.embedded[1].name)
plot!(ylabel="Electron density / m^(-3)", xlabel="Time / s", legend=true, left_margin=10Plots.pt)